Cargo desde disco el fichero baby_reviews_small.csv usando pandas y lo guardo en un DataFrame llamado df_small.


In [ ]:
import pandas as pd

df_small = pd.read_csv("/content/baby_reviews_small.csv")
df_small.head()


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,sentiment
0,A1OFPQMJWMM4XM,B002OHD3XS,"KARARED ""Kc""","[0, 0]",I love this booster. We have two and it's wel...,5,Love this booster....SAFETY FIRST!!!!!,1368403200,"05 13, 2013",1
1,A1Z94P3Q8O0D21,B00354UNSM,Jana Pettus,"[1, 1]","This has a million parts, is hard to wash, and...",1,Don't waste your money,1393891200,"03 4, 2014",0
2,A1V981XARKIMNT,B001KZJY2O,Chad and Kristen,"[1, 2]",This pillow probably cost less than a dollar t...,1,Terrible quality - do not buy this pillow,1367193600,"04 29, 2013",0
3,A1TCX0R35T4WEH,B000056J2D,"MomOf2BOYS ""Lucy""","[1, 2]","My 2 1/2 year olds highchair was terrible, so ...",5,Great booster,1115683200,"05 10, 2005",1
4,A1N54YHG0KQI91,B001PBFWMO,Toddler Mom,"[1, 1]",We got this when our daughter was only a few m...,5,We LOVE this!,1358899200,"01 23, 2013",1


Aquí preparo todas las herramientas que voy a usar para limpiar el texto de las reviews. Importo re y BeautifulSoup para eliminar HTML y caracteres raros, y uso nltk para descargar las stopwords en inglés y el PorterStemmer. Guardo las palabras vacías en stop_words y creo un objeto stemmer para reducir las palabras a su raíz. Todo esto lo utilizaré en una función de preprocesado del texto.


In [ ]:
import re
from bs4 import BeautifulSoup

import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


En este bloque defino la función preprocess, que se encarga de limpiar el texto de cada review para poder usarlo en el modelo. Primero controlo el caso en que el valor no sea un string y devuelvo una cadena vacía. Después elimino posibles etiquetas HTML, paso todo a minúsculas, borro URLs y símbolos no alfanuméricos, divido el texto en palabras, quito las stopwords en inglés y aplico stemming para reducir cada palabra a su raíz. Al final vuelvo a unir los tokens en una sola cadena limpia, que será la versión “procesada” de la review.


In [ ]:
def preprocess(text):

    if not isinstance(text, str):
        return ""

    text = BeautifulSoup(text, "html.parser").get_text()

    text = text.lower()

    text = re.sub(r"http\S+|www\S+", " ", text)

    text = re.sub(r"[^a-z0-9\s]", " ", text)

    tokens = text.split()

    tokens = [t for t in tokens if t not in stop_words]

    tokens = [stemmer.stem(t) for t in tokens]

    return " ".join(tokens)


In [ ]:
ejemplo = df_small["reviewText"].iloc[0]
print("ORIGINAL:\n", ejemplo)
print("\nPROCESADO:\n", preprocess(ejemplo))


ORIGINAL:
 I love this booster.  We have two and it's well worth the money.  I don't know of any other booster seat that actually uses latches and that is why we  went with this one.  I didn't want the booster moving around on the seat of the car.  Easy to get in and out of the car as well.  This one grows with you child.  You can remove headrest, then removed back support separately when the time comes.  You will not be disappointed.

PROCESADO:
 love booster two well worth money know booster seat actual use latch went one want booster move around seat car easi get car well one grow child remov headrest remov back support separ time come disappoint


In [ ]:
df_small["clean_text"] = df_small["reviewText"].apply(preprocess)
df_small[["reviewText", "clean_text"]].head()


,reviewText,clean_text
0,I love this booster. We have two and it's wel...,love booster two well worth money know booster...
1,"This has a million parts, is hard to wash, and...",million part hard wash even get get milk engor...
2,This pillow probably cost less than a dollar t...,pillow probabl cost less dollar manufactur mad...
3,"My 2 1/2 year olds highchair was terrible, so ...",2 1 2 year old highchair terribl threw first b...
4,We got this when our daughter was only a few m...,got daughter month old diagnos rsv absolut wou...


In [ ]:
df_small.to_csv("baby_reviews_small_clean.csv", index=False)
